In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import keras_nlp
import keras

from collections import Counter

import re
import whoosh_utils
import whoosh

import os
import gc


pd.set_option('display.max_rows', 200)
pd.options.mode.copy_on_write = True

2024-06-22 18:59:49.579932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 18:59:49.580042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 18:59:49.723073: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl


In [3]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/uspto-explainable-ai"
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    input_length = 1024 # max size of input sequence for training
    output_length = 1200 # max size of output sequence
    num_neighbors = 3 # how many neighbour patents to consider
    n_main_cpc = 2

In [4]:
#os.chdir('/kaggle/input/uspto-explainable-ai/')
os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

keras.utils.set_random_seed(CFG.seed)

print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

Keras: 3.3.3
KerasNLP: 0.12.1


In [ ]:

# test_df, где 2 соседа входят в индекс

'''
%%time
index_list = pd.read_json('/kaggle/input/uspto-explainable-ai/train_index_patent_ids.json')

nn_columns = pd.read_csv('/kaggle/input/uspto-explainable-ai/nearest_neighbors.csv', header=0, nrows = 1).columns

test_df = pd.read_csv('/kaggle/input/uspto-explainable-ai/nearest_neighbors.csv', names=nn_columns, skiprows=0, nrows = 1000000)
test_df = test_df[(test_df.neighbor_0.isin(index_list[0])) 
               & (test_df.neighbor_1.isin(index_list[0]))]

for i in range(1, 14):
    test_df1 = pd.read_csv('/kaggle/input/uspto-explainable-ai/nearest_neighbors.csv', names=nn_columns, skiprows=i * 1000000, nrows = 1000000)

    test_df = pd.concat([test_df, 
                         test_df1[(test_df1.neighbor_0.isin(index_list[0])) 
                                  & (test_df1.neighbor_1.isin(index_list[0]))
                                 ]
                        ])
    print(i, test_df.shape, test_df1.shape)
    
    
test_df.to_pickle('/kaggle/working/test_df.pkl')
'''

In [12]:
# test_df для работы n - кол-во "соседей" в индексе
# + main_cpc_codes
'''
%%time
test_df = pd.read_pickle('/kaggle/input/test-df/test_df.pkl')
#test_df = pd.read_pickle('/kaggle/working/test_df.pkl')
print(test_df.shape)

index_list = pd.read_json('/kaggle/input/uspto-explainable-ai/train_index_patent_ids.json')

test_df['nn'] = 0

for i in range(50):
    test_df.loc[test_df['neighbor_'+str(i)].isin(index_list[0].values), 'nn'] = test_df['nn'] + 1
    
test_df = test_df[test_df.nn>2]

meta_df = pd.read_parquet(f"{CFG.dataset_path}/patent_metadata.parquet")

main_cpc_codes = []
for _, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    df = meta_df[meta_df.publication_number.isin(row[:51].values)]
    cpc = df.cpc_codes.values
    
    counter_object = Counter([x for sub_list in cpc for x in sub_list])
    key_cpc = [x[0] for x in counter_object.most_common()][:CFG.n_main_cpc]
    df['cc'] = 0
    
    for code in key_cpc:
        df.loc[df['cpc_codes'].apply(lambda x: key_cpc[1] in x), 'cc'] = df['cc'] + 1
        
    df1 = df[df['cc']==0]
    while df1.shape[0] > 0:
        cpc = df1.cpc_codes.values
        counter_object = Counter([x for sub_list in cpc for x in sub_list])
        
        try:
            add_cpc = counter_object.most_common()[0][0]
            df1.loc[df1['cpc_codes'].apply(lambda x: add_cpc in x), 'cc'] = df1['cc'] + 1
            key_cpc.append(add_cpc)
            df1 = df1[df1['cc']==0]
        except:
            break
        
    main_cpc_codes.append(key_cpc)

test_df["main_cpc"] = main_cpc_codes

del meta_df
gc.collect()

print(test_df.shape, test_df.columns)
'''

100%|██████████| 2846/2846 [35:52<00:00,  1.32it/s]


In [ ]:
%%time
#uncomment before submit

test_df = pd.read_csv(f"{CFG.dataset_path}/test.csv")
#test_df = test_df.iloc[:, :CFG.num_neighbors+1]
target_cols = list(test_df.columns[1:CFG.num_neighbors+1])

# Merge metadata of the patents
meta_df = pd.read_parquet(f"{CFG.dataset_path}/patent_metadata.parquet")
test_df = test_df.merge(meta_df, on="publication_number", how="left")

# Merge Title and Abstract of the patennts
all_patents = pd.read_parquet("/kaggle/input/uspto-all-patents-after-1975/all_patents.parquet")
test_df = test_df.merge(all_patents, on="publication_number")

# Fill NaN values
test_df["title"] = test_df["title"].fillna("")
test_df["abstract"] = test_df["abstract"].fillna("")

# Merge Title and Abstract of the neighbour patents
for i in range(CFG.num_neighbors):
    test_df = test_df.merge(
        all_patents,
        left_on=target_cols[i],
        right_on="publication_number",
        how="left",
        suffixes=("", f"_{i}")
    )

    '''test_df = test_df.merge(
        meta_df[['publication_number', 'cpc_codes']],
        left_on=target_cols[i],
        right_on="publication_number",
        how="left",
        suffixes=("", f"__{i}")
    )

    # Fill NaN values
    test_df[f"cpc_codes__{i}"] = test_df[f"cpc_codes__{i}"].fillna("")'''
    
    test_df[f"title_{i}"] = test_df[f"title_{i}"].fillna("")
    test_df[f"abstract_{i}"] = test_df[f"abstract_{i}"].fillna("")

    # Drop extra publication_number column from merges
    test_df = test_df.drop(columns=[f"publication_number_{i}"])    #, f"publication_number__{i}"])

main_cpc_codes = []
for _, row in tqdm(test_df.iterrows()):
    df = meta_df[meta_df.publication_number.isin(row[:51].values)]
    cpc = df.cpc_codes.values
    
    counter_object = Counter([x for sub_list in cpc for x in sub_list])
    key_cpc = [x[0] for x in counter_object.most_common()][:CFG.n_main_cpc]
    df['cc'] = 0
    
    for code in key_cpc:
        df.loc[df['cpc_codes'].apply(lambda x: key_cpc[1] in x), 'cc'] = df['cc'] + 1
        
    df1 = df[df['cc']==0]
    while df1.shape[0] > 0:
        cpc = df1.cpc_codes.values
        counter_object = Counter([x for sub_list in cpc for x in sub_list])
        
        try:
            add_cpc = counter_object.most_common()[0][0]
            df1.loc[df1['cpc_codes'].apply(lambda x: add_cpc in x), 'cc'] = df1['cc'] + 1
            key_cpc.append(add_cpc)
            df1 = df1[df1['cc']==0]
        except:
            break       
         
    main_cpc_codes.append(key_cpc)

test_df["main_cpc"] = main_cpc_codes

test_df = test_df.reset_index(drop=True)

# Clean up memory
del meta_df, all_patents, df, df1
gc.collect()

# Model

In [13]:
%%time
# Declare the model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(CFG.preset)

# Set input length of small to keep the memory and latency cost small
gemma_lm.preprocessor.sequence_length = CFG.input_length

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Kaggle notebook...
Attachin

CPU times: user 11.4 s, sys: 14.1 s, total: 25.5 s
Wall time: 1min 8s


In [14]:
def create_prompt(row, neighbor_idx):
    #prompt = chat_template.format(title_a=row["title"], abstract_a=row["abstract"],
    #                              title_b=row[f"title_{neighbor_idx}"], abstract_b=row[f"abstract_{neighbor_idx}"])
    prompt = chat_template.format(title_a=row["title"], abstract_a="",
                                  title_b=row[f"title_{neighbor_idx}"], abstract_b="")
    return prompt


In [15]:
def generate_keyword(row, neighbor_idx):
    # Check if any title or abstract is an empty string
    fields = [
        "title",
        #"abstract",
        f"title_{neighbor_idx}",
        #f"abstract_{neighbor_idx}",
    ]
    if all(row[field] == "" for field in fields):
        return [""]

    # Create prompt
    prompt = create_prompt(row, neighbor_idx)

    try:
        output = gemma_lm.generate(prompt, max_length=CFG.output_length).replace(prompt, "").strip()
        
        for symb in ['**Keywords list:**', 'Keywords list', ':', ';', '$', '?', '.']:
            output = output.replace(symb, "")
        
        answer = output.strip().split('\n* ')

        keywords = [x.replace("*", "").strip() for x in set(answer) if len(x) < 50]     

    except:
        return [""]

    return keywords

In [16]:
prompt_template = "Request:\nAnalyze and compare the given two patent abstracts and titles, and identify the common and similar keywords that should find specifically these two patents when searched all Patent database.\n\nIn the answer provide only the list of keywords, named 'Keywords list:', separating keywords with a '\n* '. \n\nPatent 1:\n* Title: {title_a}\n* Abstract: {abstract_a}\n\nPatent 2:\n* Title: {title_b}\n* Abstract: {abstract_b}\n"

chat_template = f"<start_of_turn>user\n{prompt_template}<end_of_turn>\n<start_of_turn>model\n"

In [ ]:
#[print(f"Keywords {i}: {q}", end="\n\n") for i, q in  enumerate(keywords_all_0[:10])]

In [17]:
BRS_STOPWORDS = ['a', 'an', 'are', 'by', 'for', 'if', 'into', 'is', 'no', 'not', 'of', 'on', 'such',
        'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'thus', 'to', 'was', 'will', 'and', 'or', 'in']
NUMBER_REGEX = re.compile(r'^(\d+|\d{1,3}(,\d{3})*)(\.\d+)?$')

class NumberFilter(whoosh.analysis.Filter):
    def __call__(self, tokens):
        for t in tokens:
            if not NUMBER_REGEX.match(t.text):
                yield t

custom_analyzer = whoosh.analysis.StandardAnalyzer(stoplist=BRS_STOPWORDS) | NumberFilter()

In [18]:
query_validator = whoosh_utils.QueryValidator()

def validate_query(query):
    query = "ti:device" if not len(query) or not isinstance(query, str) else query
    try:
        query_validator.validate_query(query)
    except:
        query = "ti:device"
    return query

In [ ]:
%%time

keywords_all = []

#for i, row in tqdm(test_df[:30].iterrows(), total=test_df[:30].shape[0]):
for _, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    # Keywords for one patent
    keywords = []
    
    for j in range(CFG.num_neighbors):
        keywords += generate_keyword(row, neighbor_idx=j)
        
    #tokens = [token.text for token in custom_analyzer(' '.join(keywords))]
    #counter_object = Counter(tokens)

    keywords1 = []
    for keyword in keywords:
        keywords1.append('(' + ' AND '.join([token.text for token in custom_analyzer(keyword)][:2]) + ')')
    #print(keywords1)
    counter_object = Counter(keywords1)
    keywords = [x[0] for x in counter_object.most_common()]
    
    keywords_all.append(keywords)

  0%|          | 0/2846 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1719087215.171296      34 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719087215.256472      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1719087215.853150      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
 88%|████████▊ | 2493/2846 [3:19:45<27:16,  4.64s/it]  

In [ ]:
'''
%%time
train_idx = whoosh_utils.load_index('/kaggle/input/uspto-explainable-ai/train_index')
searcher = whoosh_utils.get_searcher(train_idx)
qp = whoosh_utils.get_query_parser()
'''

In [ ]:
%%time

#by words
queries = []
s=0
i=0

for _, row in test_df.iterrows():
#for i, row in test_df[:30].iterrows():

    '''p_cpc_codes = list(row["cpc_codes"])

    for j in range(CFG.num_neighbors):
        p_cpc_codes.extend(row[f"cpc_codes__{j}"])
    
    counter_object = Counter(p_cpc_codes)
    cpc = [x[0] for x in counter_object.most_common()]'''

    cpc = row["main_cpc"]

    query_cpc = f"cpc:({' OR '.join(cpc[:17])})" if len(cpc) else ""
    
    try:
        
        keywords = keywords_all[i]
        #keywords = '(' + ' ADJ2 '.join() + ')'
        #print('keywords', keywords)

        # number of query tokens must be <= 50
        while len(keywords)>0:
            ti_keywords = f"({' OR '.join(keywords)})"
            #detd_keywords = f"({' OR '.join(keywords)})"
            
            #query = (f"ti:{ti_keywords} OR (detd:{detd_keywords} AND " + 
            #         (f"{query_cpc}" if len(query_cpc) else '') + ")"
            #        )
            query = (f"ti:{ti_keywords} AND " +
                     #f"detd:{detd_keywords} AND " +
                     f"{query_cpc}")
            #print('counts_tokens', whoosh_utils.count_query_tokens(query))
            
            if whoosh_utils.count_query_tokens(query) >50:
                #print(whoosh_utils.count_query_tokens(query))
                keywords = keywords[:-1]
            else:
                break

    except:
        query = query_cpc
  
    #query = query_cpc
    
    # Final query validation
    query = validate_query(query)
    
    '''p_list = whoosh_utils.execute_query(query, qp, searcher)
    n = 0
    for p_num in p_list:
        if p_num in list(row[1:51]): #.values.reshape(51)[1:]:
            n += 1
    s += n
    print(i, f"{n}/{row['nn']}. Total {s}") #, len(p_list), '\n', qp.parse(query))'''
    
    queries.append(query)
    i += 1

In [41]:
'''
print(i, f"{n}/{row['nn']}. Total {s}", len(p_list), query)
print(qp.parse(query))
'''
#print('\n'.join(queries))

91 1/6. Total 0 50 cpc:(H04L63/083 OR H04W12/06 OR G06F21/31 OR H04L63/083 OR G06F21/31 OR H04L63/08 OR G06F21/88 OR G07C2009/00769 OR H04W12/126 OR G06F21/36 OR G06F21/335 OR G06F21/35 OR G06F21/34 OR G06F21/305 OR G06F3/1204)
(cpc:H04L63/083 OR cpc:H04W12/06 OR cpc:G06F21/31 OR cpc:H04L63/08 OR cpc:G06F21/88 OR cpc:G07C2009/00769 OR cpc:H04W12/126 OR cpc:G06F21/36 OR cpc:G06F21/335 OR cpc:G06F21/35 OR cpc:G06F21/34 OR cpc:G06F21/305 OR cpc:G06F3/1204)


In [ ]:
'''0 5/7. Total 5 50 cpc:(F17C2221/033 OR F17C2223/0123 OR F17C2225/0123 OR F17C2225/036 OR F17C5/007 OR F17C2201/0109 OR F17C2201/032 OR F17C5/06 OR F17C2201/054 OR F17C2223/033 OR F17C2227/0192 OR F17C2250/01 OR F17C2265/065 OR F17C2270/0139 OR F17C2270/0171)
1 2/6. Total 7 50 cpc:(G06F3/0488 OR G06F3/017 OR G06F21/32 OR H04M1/72403 OR H04M1/72522 OR H04N23/632 OR H04N5/23293 OR G02B2027/0187 OR G02B27/0093 OR G06F21/00 OR G06F3/012 OR G06F3/013 OR G06F3/0484 OR G06F3/14 OR H04M1/72409)
2 1/6. Total 8 50 cpc:(G16H40/63 OR G16H40/67 OR G16H50/20 OR A61B5/1117 OR E04F15/105 OR E04F15/107 OR E04F19/0436 OR E04F19/061 OR E04F2019/044 OR E04F2290/02 OR G06N20/00 OR G06V20/52 OR G16H40/20 OR G16H50/70 OR G16H50/80)
3 4/6. Total 12 50 cpc:(F02N15/067 OR F02N11/00 OR F02N15/022 OR F02N15/046 OR F02N15/062 OR Y10T74/132 OR F02N2015/061 OR F16D27/118 OR F02N15/023 OR F02N11/0814 OR F02N11/0851 OR F02N15/02 OR F02N15/04 OR F16D41/185 OR F02N15/065)
4 1/7. Total 13 50 cpc:(G06K9/2018 OR G06K9/2027 OR G06V10/143 OR G06T7/90 OR H04N23/11 OR H04N5/332 OR G06T2207/10048 OR G06T2207/10152 OR G06T7/11 OR G06V10/141 OR G06V40/168 OR G06K9/00355 OR G06V40/28 OR H04N23/56 OR H04N5/2226)
5 3/6. Total 16 50 cpc:(G05B15/02 OR Y02P90/82 OR G05F1/66 OR G06Q50/06 OR G06Q10/06 OR G01R21/133 OR Y02B90/20 OR Y02P90/84 OR Y02P90/845 OR H02J2203/20 OR H02J3/00 OR Y02B70/3225 OR Y02E60/00 OR Y04S20/222 OR Y04S40/20)
6 0/6. Total 16 50 cpc:(C07C69/92 OR C07C201/16 OR C07C47/56 OR C07C47/565 OR C07C47/575 OR C07C51/29 OR C07C51/363 OR C07C51/367 OR C07C51/377 OR C07C65/05 OR C07C65/21 OR C07C67/035 OR C07C67/307 OR C07C67/317 OR C07C69/84)
7 2/6. Total 18 50 cpc:(B25J19/0008 OR B25J19/0012 OR Y10S901/15 OR Y10S901/27 OR Y10T74/20305 OR B25J17/0216 OR B25J19/0091 OR B25J9/0057 OR B25J9/0075 OR B25J9/06 OR B25J9/08 OR B25J9/144 OR Y10S901/22 OR Y10S901/28 OR B66C13/20)
8 1/6. Total 19 50 cpc:(G02B2027/014 OR G06T19/006 OR G02B2027/0138 OR G02B2027/0178 OR G02B27/017 OR G06F3/012 OR G02B27/0172 OR G02B27/0093 OR G06F3/013 OR G06T2207/30244 OR H04N23/90 OR H04N5/247 OR H04N13/344 OR G02B2027/0187 OR G06F3/011)
9 2/6. Total 21 50 cpc:(G01N13/00 OR G01N1/2035 OR G01N33/2847 OR G01N11/04 OR G01N2021/8405 OR G01N21/85 OR G01N33/2823 OR G06F18/241 OR G06F18/256 OR G06K9/00134 OR G06K9/00147 OR G06K9/6268 OR G06K9/6293 OR G06V20/693 OR G06V20/698)
10 3/6. Total 24 50 cpc:(G06N3/044 OR G06N3/045 OR G06N3/0445 OR G06N3/08 OR G06N3/0454 OR G06N3/084 OR G06N3/048 OR G06F16/345 OR G06F16/93 OR G06F40/30 OR G06F40/56 OR G06F40/216 OR G06F40/279 OR G06F40/40 OR G06N3/006)
    
90 3/6. Total 296 50 cpc:(H04W12/06 OR H04L63/083 OR G06F21/31 OR G06F21/316 OR H04M1/724631 OR H04W12/068 OR H04W12/08 OR H04W88/02 OR H04W12/30 OR H04W12/64 OR H04M1/72577 OR H04W12/126 OR G06F21/30 OR G06F21/41 OR G06F2221/2113)    '''

In [ ]:
#test_df[1:2]
#test_df.nn.sum()

In [ ]:
test_df["query"] = queries
subm = test_df[["publication_number", "query"]]
subm.to_csv("submission.csv", index=False)
subm.head()